In [1]:
import os
os.environ["LOGLEVEL"] = "PROD"

import numpy as np
import pandas as pd
from neuro_dmt.models.bluebrain.circuit.model import BlueBrainCircuitModel
from neuro_dmt.models.bluebrain.circuit.mock.test.mock_circuit_light\
    import circuit_composition, circuit_connectivity
from neuro_dmt.models.bluebrain.circuit.mock.circuit import\
    MockCircuit


('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')


A few tests for `class CellType`.

We can define a `BlueBrainCircuitModel` instance for `MockCircuit`:

In [2]:
mock_circuit =\
    MockCircuit.build(
        circuit_composition,
        circuit_connectivity)
circuit_model =\
    BlueBrainCircuitModel(
        mock_circuit,
        label="BlueBrainCircuitModelMock")
conn_prob = circuit_model.connection_probability

In [4]:
pre_synaptic_cells = circuit_model.cells.sample(n=100)
post_synaptic_cells = circuit_model.cells.sample(n=100)
conn_prob.get_summary(
    pre_synaptic_cell_group=pre_synaptic_cells,
    post_synaptic_cell_group=post_synaptic_cells)

pairs_total               10000.0000
pairs_connected             206.0000
connection_probability        0.0206
dtype: float64

In [5]:
pre_type = pd.Series({"mtype": "L23_MC"})
post_type = pd.Series({"mtype": "L23_MC"})
pathway = pd.Series({
    "pre_synaptic_mtype": "L23_MC",
    "post_synaptic_mtype": "L23_MC"})
result_series = conn_prob.get_summary(
    pre_synaptic_cell_group=pre_type,
    post_synaptic_cell_group=post_type, 
    number=100)
result_series

pairs_total               10000.0000
pairs_connected             185.0000
connection_probability        0.0185
dtype: float64

In [8]:
conn_prob.get_summary(
    pre_synaptic_cell_group=pre_type,
    post_synaptic_cell_group=post_type, 
    number=100,
    resample=True)

pairs_total               10000.0000
pairs_connected             194.0000
connection_probability        0.0194
dtype: float64

In [9]:
from neuro_dmt.models.bluebrain.circuit.model.pathway import GroupByVariables
result_groupby = conn_prob(
    groupby=GroupByVariables(
        frozenset(["mtype"]),
        frozenset(["mtype"])))

In [10]:
result_groupby.head()

pairs_total  pairs_connected  \
pre_synaptic_mtype post_synaptic_mtype                                 
L1_DAC             L1_DAC                         3              0.0   
                   L1_SLAC                        3              0.0   
                   L23_ChC                        6              0.0   
                   L23_MC                         6              0.0   
                   L23_SBC                        9              0.0   

                                        connection_probability  
pre_synaptic_mtype post_synaptic_mtype                          
L1_DAC             L1_DAC                                  0.0  
                   L1_SLAC                                 0.0  
                   L23_ChC                                 0.0  
                   L23_MC                                  0.0  
                   L23_SBC                                 0.0

In [13]:
from neuro_dmt.models.bluebrain.circuit.model.pathway import GroupByVariables
result_dataframes = conn_prob(
    pre_synaptic_cell_group=pre_synaptic_cells,
    post_synaptic_cell_group=post_synaptic_cells,
    groupby=GroupByVariables(("mtype", ), ("mtype", )),
    number=100,
    resample=True)
result_dataframes

In [17]:
result_dataframes = conn_prob.get_summary(
    pre_synaptic_cell_group=pre_synaptic_cells,
    post_synaptic_cell_group=post_synaptic_cells,
#    groupby=GroupByVariables(("mtype", ), ("mtype", )),
    number=100,
    resample=True)
result_dataframes

pairs_total               10000.0000
pairs_connected             222.0000
connection_probability        0.0222
dtype: float64

In [11]:
result_no_cell_groups = conn_prob(
    groupby=GroupByVariables(("mtype", ), ("mtype", )),
    resample=True,
    number=100)

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 21:28:19>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	334
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            Call to connection probability.
            pathway:
            	None
            pre_synaptic_cell_group:
            	None
            post_synaptic_cell_group:
            	None
            to be group by:
            	GroupByVariables(pre_synaptic_cell_type_specifier=('mtype',), post_synaptic_cell_type_specifier=('mtype',))
            methodology:
            	random
            

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 21:28:19>                                                     
Traceback:
	filename: 

In [12]:
print(result_no_cell_groups.sum())
result_no_cell_groups.head()

pairs                   total        10000.000000
                        connected      201.000000
connection_probability                   5.106456
dtype: float64


pairs           connection_probability
                                       total connected                       
pre_synaptic_mtype post_synaptic_mtype                                       
L1_SLAC            L1_DAC                  1       0.0                    0.0
                   L1_SLAC                 1       0.0                    0.0
                   L23_ChC                 1       0.0                    0.0
                   L23_MC                  2       0.0                    0.0
                   L23_SBC                 2       0.0                    0.0

In [13]:
pre_synaptic = pd.Series({"mtype": "L6_UPC"})
post_synaptic = pd.Series({"mtype": "L6_UPC"})
pathway = CellType.pathway(pre_synaptic, post_synaptic)
result_no_cell_groups.head()

pairs           connection_probability
                                       total connected                       
pre_synaptic_mtype post_synaptic_mtype                                       
L1_SLAC            L1_DAC                  1       0.0                    0.0
                   L1_SLAC                 1       0.0                    0.0
                   L23_ChC                 1       0.0                    0.0
                   L23_MC                  2       0.0                    0.0
                   L23_SBC                 2       0.0                    0.0

In [14]:
levels_and_values = [
    ('_'.join(k), v) for k, v in zip(pathway.index.values, pathway.values)]
levels = tuple('_'.join(k) for k in pathway.index.values)
values = pathway.values

result_no_cell_groups.xs(
    values,
    level=levels)


pairs           connection_probability
                                       total connected                       
pre_synaptic_mtype post_synaptic_mtype                                       
L6_UPC             L6_UPC                150       2.0               0.013333

In [15]:
result_no_cell_groups_no_types = conn_prob(number=1000)

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 21:28:19>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	334
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            Call to connection probability.
            pathway:
            	None
            pre_synaptic_cell_group:
            	None
            post_synaptic_cell_group:
            	None
            to be group by:
            	GroupByVariables(pre_synaptic_cell_type_specifier=None, post_synaptic_cell_type_specifier=None)
            methodology:
            	random
            

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 21:28:19>                                                     
Traceback:
	filename: 	/home/muchu

In [16]:
result_no_cell_groups_no_types

total        1000000
connected      19473
dtype: int64

Test pathway properties in circuit analysis, using the mock circuit.

In [1]:
from neuro_dmt.models.bluebrain.circuit.adapter import BlueBrainCircuitAdapter
from neuro_dmt.models.bluebrain.circuit.test.test_connectome_analysis import *
phenomenon = Phenomenon(
    "Connection Probability",
    """
    Probability that two neurons in a pathway are connected.
    While mostly interested in `mtype->mtype` pathways, we can define
    a pathway as the group of connected neurons with pre-synaptic and
    post-synaptic neurons belonging to two parametrically defined groups.
    As an example consider neuron populations defined by not only `mtype`
    but also by soma-distance from a given location.
    """,
    group="Connectome")
pathways = mock_circuit_model.pathways(frozenset({"mtype", })).sample(n=5)
analysis = BrainCircuitAnalysis(
    phenomenon=phenomenon,
    AdapterInterface=ConnectionProbabilityInterface,
    measurement_parameters=Parameters(pathways),
        plotter=HeatMap(
        xvar=("pre_synaptic", "mtype"),
        xlabel="pre-mtype",
        yvar=("post_synaptic", "mtype"),
        ylabel="post-mtype",
        vvar=("connection_probability", "mean")))
analysis_test = BlueBrainCircuitAnalysisTest(
    analysis=analysis)
adapter = BlueBrainCircuitAdapter()
analysis_test.test_circuit_model(circuit_label)
measurement = analysis.get_measurement(mock_circuit_model, adapter, sample_size=100)
#cpdf = list(measurement.values())[0]
#cpdf.head()


('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')


In [7]:
parameters = analysis.measurement_parameters
[adapter.get_connection_probability(mock_circuit_model, **p)
 for p in parameters.for_sampling(size=1)]

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	334
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            Call to connection probability.
            pathway:
            	pre_synaptic   mtype    L6_MC
		post_synaptic  mtype    L6_MC
		dtype: object
            pre_synaptic_cell_group:
            	None
            post_synaptic_cell_group:
            	None
            to be group by:
            	GroupByVariables(pre_synaptic_cell_type_specifier=None, post_synaptic_cell_type_specifier=None)
            methodology:
            	random
            

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>               

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	124
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            resolve cell group mtype    L1_SLAC
dtype: object
            given that {'sampling_methodology': 'random', 'resample': False, 'number': 100}
            

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	124
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            resolve cell group {'mtype': 'L1_SLAC'

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	334
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            Call to connection probability.
            pathway:
            	pre_synaptic   mtype    L6_TPC:A
		post_synaptic  mtype      L4_LBC
		dtype: object
            pre_synaptic_cell_group:
            	None
            post_synaptic_cell_group:
            	None
            to be group by:
            	GroupByVariables(pre_synaptic_cell_type_specifier=None, post_synaptic_cell_type_specifier=None)
            methodology:
            	random
            

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>         

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	124
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            resolve cell group {'mtype': 'L6_MC'}
            given that {'sampling_methodology': 'random', 'resample': False, 'number': 100}
            

/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py Logger
FUNDA@<2019-11-06 22:24:32>                                                     
Traceback:
	filename: 	/home/muchu/work/bbp/work/validations/dmt/v2/neuro_dmt/models/bluebrain/circuit/model/pathway.py
	lineno: 	124
	code_context: 	['            logger.get_source_info(),\n']
	index: 0


            resolve cell group         mtype    gid
0      L1_

[0.0, 0.0, nan, nan, 0.0]

In [5]:
from neuro_dmt.models.bluebrain.circuit.model.pathway import GroupByVariables
result = conn_prob(
        pre_synaptic_cell_group=pre_synaptic_cells,
        post_synaptic_cell_group=post_synaptic_cells,
        groupby=GroupByVariables(
            pre_synaptic_cell_type_specifier={"mtype",},
            post_synaptic_cell_type_specifier={"mtype",}))
pre_synaptic_cells.shape, result.shape

group summary


((100, 11), (256, 3))

In [7]:
result = conn_prob(
        pre_synaptic_cell_group=pre_synaptic_cells,
        post_synaptic_cell_group=post_synaptic_cells)
result

0.0203

In [12]:
from neuro_dmt.models.bluebrain.circuit.model.test.pathway import *
test_call_with_dataframes_and_groupby()

KeyError: "['pre_synaptic_mtype', 'post_synaptic_mtype'] not in index"

## Pathway property

A `PathwayProperty` instance is a callable -- and callable:
```
class PathwayProperty(WithFields):
    """
    Compute and store a circuit's pathway properties.
    """
    phenomenon = Field(
        """
        A label to go with the pathway property.
        """)
    circuit_model = Field(
        """
        The circuit model for which this `PathwayProperty` has been defined.
        """)
    max_length_cell_type_specification = Field(
        """
        Max length of the cell type specifier that can be stored.
        """,
        __default_value__=2)

    def __call__(self,
            pre_synaptic_cell_group,
            post_synaptic_cell_group,
            by=None,
            given=None,
            resample=False,
            sampling_methodology=SamplingMethodology.random,
            number=100):
        """
        Compute, or retrieve from the store...

        Arguments
        -------------
        pre_synaptic_cell_group:  Array<GID>,
        ~                         Mapping<CellProperty==>Value>,
        ~                         pandas.Series<CellProperty>,
        ~                         pandas.DataFrame<CellProperty>
        post_synaptic_cell_group: Array<GID>,
        ~                         Mapping<CellProperty==>Value>,
        ~                         pandas.Series<CellProperty>,
        ~                         pandas.DataFrame<CellProperty>
        by : A property of a connection ((pre_synaptic_cell, post_synaptic_cell).
             Pathway property will be computed for given values of this variable.
             If a list of variables, the computation will be conditioned on all
             the variables.
        given: A dict providing variables and their values for which the
               the pathway property will be returned.
               
        Returns
        ---------------
        pd.Series if 
        
        """
```



In [ ]:
s = circuit_model.connection_probability(
    np.array([1,2]),
    np.array([3,4]))

In [ ]:
s

In [ ]:
from neuro_dmt.models.bluebrain.circuit.model.pathway import SamplingMethodology
pre_cell_group = pd.Series({"mtype": "L23_MC"})
post_cell_group = pd.Series({"mtype": "L23_MC"})
s = circuit_model.connection_probability(
    pre_cell_group,
    post_cell_group,
    sampling_methodology=SamplingMethodology.exhaustive)

In [ ]:
mock_circuit.cells.get({"mtype": "L23_ChC"}).head()

In [ ]:
mock_circuit.cells.get(mock_circuit.connectome.afferent_gids(140)).head()

In [ ]:
s.xs("L1_DAC", level="post_synaptic_mtype")

In [ ]:
l1dacs = mock_circuit.cells.get({"mtype": "L1_DAC"}).index.values

In [ ]:
l1dac_affs = np.concatenate(
    [mock_circuit.connectome.afferent_gids(post_gid)
     for post_gid in l1dacs],
    axis=0)

In [ ]:
l1dac_aff_mtypes = mock_circuit.cells.get(l1dac_affs, ["mtype"])

In [ ]:
l1dac_aff_mtypes.mtype.value_counts()

In [ ]:
mock_circuit.cells.get().mtype.value_counts()

In [ ]:
127890 / 70

In [ ]:
circuit_model.get_connection_probability(
    pre_cell_group, post_cell_group)

In [ ]:
def get_connection_probability(
            pre_cell_type_specifier,
            post_cell_type_specifier):
        """
        Compute connection probability once for either the entire
        circuit, or samples of afferent, or samples of  efferent,
        or samples of both efferent and afferent cells.
        """
        labels_pre_specifier =[
            "pre_synaptic_{}".format(c)
            for c in pre_cell_type_specifier]
        labels_post_specifier = [
            "post_synaptic_{}".format(c)
            for c in post_cell_type_specifier]
        cells = circuit_model.cells
        def _get_summary_connections(post_cell):
            return cells[
                list(pre_cell_type_specifier)
            ].rename(
                columns=lambda c: "pre_synaptic_{}".format(c)
            ).assign(
                number_pairs=np.in1d(
                    circuit_model.cells.index.values,
                    circuit_model.connectome.afferent_gids(post_cell.gid))
            ).groupby(
                labels_pre_specifier
            ).agg(
                ["size", "sum"]
            ).rename(
                columns={"size": "total", "sum": "connected"}
            ).assign(**{
                "post_synaptic_{}".format(p): post_cell[p] 
                for p in post_cell_type_specifier}
            ).reset_index(
            ).set_index(list(
                labels_pre_specifier + labels_post_specifier)
            )

        def _connection_probability(summary):
            """
            Compute connection probability between pairs
            """
            return summary.number_pairs.connected / summary.number_pairs.total

        return pd.concat(
            [_get_summary_connections(post_cell)
             for _, post_cell in cells.iterrows()]
        ).groupby(list(
            labels_pre_specifier + labels_post_specifier)
        ).agg(
            "sum"
        ).assign(
            connection_probability=_connection_probability )


In [ ]:
#sdirect = get_connection_probability(("mtype", ), ("mtype", ))
#sdirect.xs("L1_DAC", level="post_synaptic_mtype")

In [ ]:
l1dac_aff_mtypes.mtype.value_counts()